In [16]:
import pandas as pd
import glob
import os
import re
from pathlib import Path
import glob
from tqdm import tqdm

# I CONFIGURAÇÕES

In [27]:
# Defina os caminhos possíveis aqui
path_jorge = fr"G:\.shortcut-targets-by-id\1UEbcW3gnD82DQPOVUL7QL1Eitp5Qp_rx\Correções TCC\Tcc_aplicação\data"
path_elaine = "C:/Users/Elaine/Desktop/TCC_acidentes/data/" 


if os.path.exists(path_jorge):
    data_dir = path_jorge
    print(f"Ambiente detectado: Orientador (G:)")
elif os.path.exists(path_elaine):
    data_dir = path_elaine
    print(f"Ambiente detectado: Orientanda")
else:
    data_dir = "./dados" 
    print("Nenhum caminho absoluto encontrado. Tentando pasta local './dados'")

print(f"Lendo dados de: {data_dir}")

Ambiente detectado: Orientador (G:)
Lendo dados de: G:\.shortcut-targets-by-id\1UEbcW3gnD82DQPOVUL7QL1Eitp5Qp_rx\Correções TCC\Tcc_aplicação\data


# 1 ABERTURA DOS DADOS

In [28]:
caminho = fr'{data_dir}\ac_por_pessoa\*.csv'
arquivos = glob.glob(caminho)

print(arquivos)
print(len(arquivos), "arquivos encontrados")

['G:\\.shortcut-targets-by-id\\1UEbcW3gnD82DQPOVUL7QL1Eitp5Qp_rx\\Correções TCC\\Tcc_aplicação\\data\\ac_por_pessoa\\acidentes2019_todas_causas_tipos.csv', 'G:\\.shortcut-targets-by-id\\1UEbcW3gnD82DQPOVUL7QL1Eitp5Qp_rx\\Correções TCC\\Tcc_aplicação\\data\\ac_por_pessoa\\acidentes2020_todas_causas_tipos.csv', 'G:\\.shortcut-targets-by-id\\1UEbcW3gnD82DQPOVUL7QL1Eitp5Qp_rx\\Correções TCC\\Tcc_aplicação\\data\\ac_por_pessoa\\acidentes2021_todas_causas_tipos.csv']
3 arquivos encontrados


## 1.1 PRF

In [29]:
# Caminho onde estão os arquivos de acidentes
path_acidentes = fr'{data_dir}\ac_por_pessoa\*.csv'

lista_acidentes = []

for arquivo in tqdm(glob.glob(path_acidentes)):
    
    # Lê o CSV
    df = pd.read_csv(arquivo, encoding="ISO-8859-1", sep=";")
    
    # Filtrar colunas que você precisa
    df = df[['uf', 'br', 'km', 'municipio', 'dia_semana', 'fase_dia', 'sentido_via',
             'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
             'tipo_veiculo', 'data_inversa', 'estado_fisico']]
    
    # Converte data
    df["data_inversa"] = pd.to_datetime(df["data_inversa"], errors="coerce")
    
    # Extrair ano
    df["ano"] = df["data_inversa"].dt.year
    
    lista_acidentes.append(df)

# Concatena tudo em um dataframe só
prf = pd.concat(lista_acidentes, ignore_index=True)
prf.head()

100%|██████████| 3/3 [00:05<00:00,  1.80s/it]


,uf,br,km,municipio,dia_semana,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,data_inversa,estado_fisico,ano
0,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,Lesões Leves,2019
1,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,Lesões Leves,2019
2,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,Lesões Leves,2019
3,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,Lesões Leves,2019
4,PR,373.0,"177,3",PONTA GROSSA,terça-feira,Plena Noite,Crescente,Nublado,Dupla,Reta,Sim,Motoneta,2019-01-01,Lesões Leves,2019


## 1.2 Frota

In [32]:
# path_frota = r"C:\Users\Elaine\Desktop\TCC_acidentes\data\frota\*.xls"
path_frota = fr'{data_dir}\frota\*.xls'

lista_frotas = []

def extrair_ano_primeira_linha(arquivo):
    """
    Lê apenas a primeira linha do arquivo XLS como texto
    e extrai o ano (4 dígitos) que aparece no final.
    """
    # lê SOMENTE a primeira linha como texto bruto
    df_tmp = pd.read_excel(arquivo, header=None, nrows=1, dtype=str)
    linha = str(df_tmp.iloc[0,0])

    # pega 4 dígitos no final
    ano = re.findall(r'(\d{4})', linha)
    if ano:
        return int(ano[-1])
    else:
        raise ValueError(f"Não foi possível extrair o ano do arquivo: {arquivo}")

# leitura das tabelas de frota
for arquivo in tqdm(glob.glob(path_frota)):

    ano = extrair_ano_primeira_linha(arquivo)

    # agora lê normalmente, pulando cabeçalho
    df = pd.read_excel(arquivo, skiprows=3)

    df = df.rename(columns={"UF": "uf", "MUNICIPIO": "municipio", "TOTAL": "frota"})
    df["ano"] = ano

    # mantém apenas o necessário
    df = df[["uf", "municipio", "frota", "ano"]]

    lista_frotas.append(df)

# concatena
frota = pd.concat(lista_frotas, ignore_index=True)
frota.head()


100%|██████████| 3/3 [00:01<00:00,  2.24it/s]


,uf,municipio,frota,ano
0,AC,ACRELANDIA,7204,2021
1,AC,ASSIS BRASIL,2075,2021
2,AC,BRASILEIA,11294,2021
3,AC,BUJARI,2442,2021
4,AC,CAPIXABA,2497,2021


# 2 ACIDENTES E FROTA

In [39]:
ac_frota = prf.merge(
    frota,
    on=["municipio", "uf", "ano"],
    how="left"
)

ac_frota['gravidade'] = ac_frota['estado_fisico'].apply(
    lambda x: 1 if x in ['Lesões Graves', 'Óbito'] else 0
)

del ac_frota['estado_fisico']

print(ac_frota['gravidade'].value_counts(normalize=True))
ac_frota.head()

gravidade
0    0.841888
1    0.158112
Name: proportion, dtype: float64


,uf,br,km,municipio,dia_semana,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,data_inversa,ano,frota,gravidade
0,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,2019,705649.0,0
1,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,2019,705649.0,0
2,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,2019,705649.0,0
3,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,2019,705649.0,0
4,PR,373.0,"177,3",PONTA GROSSA,terça-feira,Plena Noite,Crescente,Nublado,Dupla,Reta,Sim,Motoneta,2019-01-01,2019,220391.0,0


In [41]:
amostra = ac_frota.sample(frac=0.10, random_state=42)

print(amostra['gravidade'].value_counts(normalize=True))
amostra.head()

gravidade
0    0.842731
1    0.157269
Name: proportion, dtype: float64


,uf,br,km,municipio,dia_semana,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,data_inversa,ano,frota,gravidade
918592,RO,364.0,"12,1",VILHENA,sexta-feira,Pleno dia,Decrescente,Céu Claro,Simples,Interseção de Vias,Sim,Motoneta,2021-07-09,2021,71697.0,0
1003548,MA,10.0,"332,2",ACAILANDIA,sexta-feira,Pleno dia,Decrescente,Céu Claro,Simples,Reta,Sim,Utilitário,2021-09-17,2021,47507.0,0
431995,PI,343.0,"341,6",TERESINA,segunda-feira,Pleno dia,Crescente,Nublado,Múltipla,Rotatória,Sim,Caminhão-trator,2020-04-27,2020,524976.0,0
452182,BA,116.0,"419,4",FEIRA DE SANTANA,sexta-feira,Anoitecer,Decrescente,Nublado,Dupla,Reta,Sim,Motocicleta,2020-05-22,2020,303018.0,0
883886,SP,116.0,211,GUARULHOS,sexta-feira,Plena Noite,Decrescente,Nublado,Múltipla,Reta,Sim,Motocicleta,2021-05-14,2021,743095.0,0


# 3 EXPORT

In [42]:
ac_frota.to_csv(Path(data_dir, "tratados/acidentes_frota_19-21.csv"), index=False)
amostra.to_csv(Path(data_dir, "tratados/amostra_19-21.csv"), index=False)